In [11]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [12]:
df15 = df15.apply(pd.to_numeric, errors = 'coerce')
df15.describe().iloc[:, 14:]
# 없는 Column = date, codesum, station_nbr

,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,97236.000000,96792.000000,97125.000000,97125.000000,97125.000000,97458.000000,97458.000000,97458.000000,97458.0,97458.0,97458.000000,97458.000000,97458.000000,97458.000000
mean,29.623938,30.007741,4.252800,22.970286,5.242514,0.417995,0.578588,0.092255,19.0,15.0,56.000000,0.297944,3.005695,0.288155
std,0.221970,0.224764,3.346494,8.996759,3.365825,0.493232,0.493788,0.289387,0.0,0.0,32.041804,2.382280,2.005405,0.452906
min,28.840000,29.220000,0.100000,1.000000,0.100000,0.000000,0.000000,0.000000,19.0,15.0,1.000000,0.000000,0.000000,0.000000
25%,29.470000,29.857500,1.700000,16.000000,2.700000,0.000000,0.000000,0.000000,19.0,15.0,28.000000,0.000000,1.000000,0.000000
50%,29.620000,30.010000,3.400000,26.000000,4.400000,0.000000,1.000000,0.000000,19.0,15.0,56.000000,0.000000,3.000000,0.000000
75%,29.780000,30.160000,6.000000,31.000000,7.100000,1.000000,1.000000,0.000000,19.0,15.0,84.000000,0.000000,5.000000,1.000000
max,30.350000,30.750000,20.800000,36.000000,21.900000,1.000000,1.000000,1.000000,19.0,15.0,111.000000,74.000000,6.000000,1.000000


In [13]:
df15['store_nbr'].unique()

array([19], dtype=int64)

In [14]:
df15_drop_columns = ['date', 'codesum', 'station_nbr']
df15 = df15.drop(columns = df15_drop_columns)

In [15]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df15_columns = df15.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df15_columns:
    if (i == 'resultdir'):
        df15[i].fillna(df15[i].mode()[0], inplace=True)
        print(df15[i].mode()[0])
    else:
        df15[i].fillna(df15[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df15['relative_humility'] = 100*(np.exp((17.625*((df15['dewpoint']-32)/1.8))/(243.04+((df15['dewpoint']-32)/1.8)))/np.exp((17.625*((df15['tavg']-32)/1.8))/(243.04+((df15['tavg']-32)/1.8))))

# 체감온도 계산
df15["windchill"] = 35.74 + 0.6215*df15["tavg"] - 35.75*(df15["avgspeed"]**0.16) + 0.4275*df15["tavg"]*(df15["avgspeed"]**0.16)

df15 = df15[df15['units'] != 0]

31.0


In [16]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df15 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + sealevel + snowfall + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df15)
result_df15 = model_df15.fit()

print(result_df15.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     227.7
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        18:47:13   Log-Likelihood:                -1963.0
No. Observations:                2868   AIC:                             3986.
Df Residuals:                    2838   BIC:                             4165.
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(nothing)[0]         0.1245      0.07

In [17]:
model_df15 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(snowfall)+ scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df15)
result_df15 = model_df15.fit()

print(result_df15.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     227.7
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        18:47:18   Log-Likelihood:                -1963.0
No. Observations:                2868   AIC:                             3986.
Df Residuals:                    2838   BIC:                             4165.
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [18]:
anova_result_df15 = sm.stats.anova_lm(result_df15).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df15[anova_result_df15['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(tmin),1.0,1.247057,1.247057,5.361397,2.065825e-02
scale(dewpoint),1.0,1.251728,1.251728,5.381483,2.042219e-02
scale(tmax),1.0,1.482787,1.482787,6.374861,1.162888e-02
scale(windchill),1.0,1.761643,1.761643,7.573727,5.960165e-03
scale(sealevel),1.0,2.175681,2.175681,9.353778,2.246032e-03
C(other),1.0,2.452577,2.452577,10.544221,1.179089e-03
scale(heat),1.0,3.365810,3.365810,14.470429,1.453799e-04
C(week7),6.0,46.468775,7.744796,33.296746,5.316247e-39
C(item_nbr),7.0,1472.795573,210.399368,904.557638,0.000000e+00
C(nothing),2.0,12082.944888,6041.472444,25973.747490,0.000000e+00


In [19]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df15.values, i) for i in range(df15.shape[1])]
vif["features"] = df15.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# C(other), C(week7), C(item_nbr)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1386: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,VIF Factor,features
0,0.000000,store_nbr
1,1.274624,snowfall
2,1.328932,other
3,1.419620,item_nbr
4,1.440670,units
5,1.475658,resultdir
6,1.614750,preciptotal
7,2.189579,depart
8,2.784603,weekend
9,2.794562,week7


In [20]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
model_df15 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(other) + scale(resultspeed) + scale(preciptotal) + \
+ scale(avgspeed) + C(weekend) + 0', data = df15)
result_df15 = model_df15.fit()

print(result_df15.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     525.8
Date:                Thu, 05 Jul 2018   Prob (F-statistic):               0.00
Time:                        18:47:35   Log-Likelihood:                -2014.3
No. Observations:                2868   AIC:                             4055.
Df Residuals:                    2855   BIC:                             4132.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(item_nbr)[15]        2.0754      0

In [21]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
X15 = df15[['resultspeed', 'avgspeed', 'weekend', 'preciptotal', 'other', 'item_nbr']]
y15 = df15['units']
model15 = LinearRegression()

cv15 = KFold(n_splits=15, shuffle=True, random_state=0)

cross_val_score(model15, X15, y15, scoring="r2", cv=cv15)

array([0.27893473, 0.32470941, 0.260441  , 0.35187391, 0.23026176,
       0.26635357, 0.26715833, 0.30097594, 0.29461317, 0.34179545,
       0.30850405, 0.21532149, 0.24493128, 0.34500232, 0.23646139])